In [7]:
import numpy as np
import gym # tested with gym version 0.12.1
from collections import deque

## Intro to Cartpole

Cartpole is a game in which a pole is attached by a joint to a cart, which moves along a frictionless track. The pole starts slightly angled, and the goal is to prevent it from falling over by applying force to the left or right of the cart.

If you can stay upright (+/- 12 degrees) for 200 time steps you win!

The game is considered solved by an AI agent when the average reward is greater than or equal to 195 over 100 consecutive trials.

For more details about the game see the [game wiki](https://github.com/openai/gym/wiki/CartPole-v0) and you can play it online [here](https://fluxml.ai/experiments/cartPole/). It's surprisingly difficult!

<img src='assets/cartpole.gif' width='25%'>

## Intro to Open AI Gym
 
[Gym](https://gym.openai.com/) is a python framework for developing and comparing reinforcement learning algorithms. In addition to Cartpole it provides several environments including Atari games like Pong and Pacman. It provides a simple interface which takes in outputs from an AI agent, and updates the environment accordingly. Leaving you to just focus on the AI agent itself!

To instantiate an environment simply use `gym.make()` and provide the name of a supported environment:

In [8]:
env = gym.make('CartPole-v0')

### Random Agent

Here's a demonstration of a very simple AI agent. The agent uses the environment's `action_space.sample()` method which simply selects an action at random from the space of possible actions (just 'left' or 'right' for the cartpole environment).

The action is then fed to the environment using using `env.step()` which returns four values:
- **observation** (object): an environment-specific object representing the new state of the environment after taking the specified action
- **reward** (float): amount of reward achieved by the previous action
- **done** (boolean): indicates whether the episode has terminated because you won or lost
- **info** (dict): diagnostic information useful for debugging

At any point we can visualize the current state of the environment by calling `env.render()`

For more information on the gym API see the official [documentation](https://gym.openai.com/docs/).

This 'agent-environment' loop continues until the the episode terminates, at which point we reset the environment and start the loop again. This loop is visualized below:

<img src="assets/agent-environment-loop.png" alt="Agent Envirnoment Loop">
<sub>Image Credit: *Reinforcement Learning: An Introduction 2nd Edition, Richard S. Sutton and Andrew G. Barto*</sub>

In [9]:
NUM_EPISODES=20
for i in range(20):
  env.reset()
  reward_sum = 0
  while True:
    #env.render()
    observation, reward, done, info = env.step(env.action_space.sample())
    reward_sum += reward
    if done:
      print("Episode {} finished with reward {}".format(i+1,reward_sum))
      break

env.close()

Episode 1 finished with reward 13.0
Episode 2 finished with reward 16.0
Episode 3 finished with reward 28.0
Episode 4 finished with reward 57.0
Episode 5 finished with reward 16.0
Episode 6 finished with reward 11.0
Episode 7 finished with reward 15.0
Episode 8 finished with reward 23.0
Episode 9 finished with reward 27.0
Episode 10 finished with reward 27.0
Episode 11 finished with reward 131.0
Episode 12 finished with reward 19.0
Episode 13 finished with reward 23.0
Episode 14 finished with reward 24.0
Episode 15 finished with reward 20.0
Episode 16 finished with reward 24.0
Episode 17 finished with reward 16.0
Episode 18 finished with reward 22.0
Episode 19 finished with reward 25.0
Episode 20 finished with reward 11.0


## Policy Network

Clearly that agent isn't going to get any better with time.

Now instead of sampling our action randomly, let's have our action be selected by a neural network.

It's input will be the observation object (state) for a given timestep, and it's output will be a number between 0 and 1 which we will interpret as the probability of applying force to the right. 

In [10]:
def sigmoid(a):
    return 1/(1+np.exp(-a)) # squashes a between 0 and 1

def policy_network(observation,model):
    """
    Arguments:
      observation: a float vector describing the environment's state
      model: a dictionary containing weight matrices 'W1','W2'
    Returns: probabilty of action '1', h
    """
    observation = np.reshape(observation,[1,len(observation)])
    h = np.matmul(observation,model['W1'])
    h[h<0] = 0 # relu 
    logit_p = np.matmul(h,model['W2'])
    return sigmoid(logit_p), h # h is returned because it will be used in backprop calculation later

### How Learning Happens

So we have our inputs and our model which allows to make predictions. But of course we don't have labels, which is what makes this a reinforcment learning problem and not a supervised learning problem.

So how do we know how to update our model weights to improve performance?

We *engineer* labels based on some policy. 

Our policy will be so simple it's surprising how well it works. 

- If the episode ends in failure, assume every action in the episode was bad. 
- If the episode ends in success, assume every action in the episode was good

We will make one improvement to this policy:

- Assume recent actions are more responsible for success/failure than earlier action

Mathematically we can accomplish this by exponentially decaying as in the following function:

In [11]:
def discount_reward(rewards,decay_rate):
    """
    Arguments:
      rewards: list of numbers
      decay_rate: rate of exponential decay
    Returns: exponentially decaying list
    """
    for i in range(len(rewards)):
        rewards[-i-1] = rewards[-i-1]*(decay_rate**i)
    return rewards    

## Training Loop

Inner loop (agent-environment loop):

1. Give state to agent, get action   
2. Give action to environment, get new state
3. Repeat until episode terminates 

Outer loop (gradient descent loop):

1. Engineer labels for each action in the episode based on whether it ended in success or failure
2. Back Propogate to calculate gradients
3. Update weights

Note how similar this is to supervised learning. In fact **it is** supervised learning, with just two differences!

1. We have to wait till an episode terminates to get our labels
2. Our labels are inferred from whether an episode fails or succeeds, as opposed to being 'ground truth'

In [13]:
%%time
# Hyperparameters
LR = 1e-1 # learning rate
HIDDEN_UNITS = 25
SEED=0 # for reproducibility
MAX_NUM_EPISODES = 5000
N_INPUTS = 4
RENDER=False
DECAY_RATE = .9

# Initialize environment
env = gym.make('CartPole-v0')
env.seed(SEED)
scores = deque(maxlen=100) # only track 100 most recent scores

# Initialize weights
np.random.seed(SEED)
model = {}
model['W1'] = np.random.randn(N_INPUTS,HIDDEN_UNITS)/np.sqrt(N_INPUTS) # xavier initialization
model['W2'] = np.random.randn(HIDDEN_UNITS,1)/np.sqrt(HIDDEN_UNITS) # xavier initialization

for i in range(MAX_NUM_EPISODES):
  # Reset environment
  observation = env.reset()
  reward_sum = 0
  episode_observations = [] # collect observations for episode
  episode_actions_p = [] # collect action probabilities for episode
  episode_actions = [] # action is also assumed label
  episode_h = [] # cache for backprop
  
  if i >= 100 and np.mean(scores) >= 195: break # solved!

  while True:
    if RENDER: env.render()
        
    # Give state to agent, get action  
    action_p, h = policy_network(observation,model) # return action probability
    action = 0 if np.random.uniform() > action_p else 1 # flip biased coin to get action (this bit of stochasticity helps with environment exploration)
    
    # Store values for this time step
    episode_observations.append(observation)
    episode_actions_p.append(action_p)
    episode_actions.append(action)
    episode_h.append(h) # cache h for manual backprop later
    
    # Give action to environment, get new state and reward
    observation, reward, done, info = env.step(action)
    reward_sum += reward
    
    # When episode terminates...
    if done: 
      # Convert python lists into matrices
      episode_observations = np.vstack(episode_observations) # EPISODE_LENGTHxN_INPUTS
      episode_actions = np.vstack(episode_actions) # EPISODE_LENGTHx1
      episode_actions_p = np.vstack(episode_actions_p) # EPISODE_LENGTHx1
      episode_h = np.vstack(episode_h) # EPISODE_LENGTHxHIDDEN_UNITS
    
      # if action is 0 want to nudge down, if action is 1 nudge up
      improve_directions = (episode_actions - episode_actions_p) 
      
      # Exponentially Decay Rewards so that most recent actions are blamed more
      improve_directions = discount_reward(improve_directions,DECAY_RATE) 
      
      # Back Propogation 
      dw2 = np.sum(h*improve_directions,axis=0,keepdims=True).T # HIDDEN_UNITSx1
      dh = np.tile(model['W2'].T,[len(episode_observations),1])*improve_directions # EPISODE_LENGTHxHIDDEN_UNITS
      dh[np.array(episode_h)<=0] = 0 # relu backprop
      dw1 = np.matmul(episode_observations.T,dh) # (N_INPUTS,EPISODE_LENGTH) x (EPISODE_LENGTH,HIDDEN_UNITS) = (N_INPUTS,HIDDEN_UNITS)
    
      # Weight Updates
      if reward_sum == 200: 
          # do more of what worked
          model['W1'] += dw1*LR
          model['W2'] += dw2*LR
      else: 
          # do less of what didn't
          model['W1'] -= dw1*LR
          model['W2'] -= dw2*LR
        
      # Log progress
      scores.append(reward_sum) # dequeue object only keeps last 100 scores
      print("Episode {} finished with reward {}. Running Avg {}".format(i+1,reward_sum,np.mean(scores)))
      
      break

Episode 1 finished with reward 30.0. Running Avg 30.0
Episode 2 finished with reward 12.0. Running Avg 21.0
Episode 3 finished with reward 17.0. Running Avg 19.666666666666668
Episode 4 finished with reward 12.0. Running Avg 17.75
Episode 5 finished with reward 12.0. Running Avg 16.6
Episode 6 finished with reward 45.0. Running Avg 21.333333333333332
Episode 7 finished with reward 42.0. Running Avg 24.285714285714285
Episode 8 finished with reward 32.0. Running Avg 25.25
Episode 9 finished with reward 30.0. Running Avg 25.77777777777778
Episode 10 finished with reward 19.0. Running Avg 25.1
Episode 11 finished with reward 9.0. Running Avg 23.636363636363637
Episode 12 finished with reward 30.0. Running Avg 24.166666666666668
Episode 13 finished with reward 45.0. Running Avg 25.76923076923077
Episode 14 finished with reward 32.0. Running Avg 26.214285714285715
Episode 15 finished with reward 44.0. Running Avg 27.4
Episode 16 finished with reward 21.0. Running Avg 27.0
Episode 17 finishe

Episode 146 finished with reward 200.0. Running Avg 127.93
Episode 147 finished with reward 200.0. Running Avg 128.35
Episode 148 finished with reward 200.0. Running Avg 128.35
Episode 149 finished with reward 156.0. Running Avg 129.26
Episode 150 finished with reward 200.0. Running Avg 130.52
Episode 151 finished with reward 200.0. Running Avg 130.52
Episode 152 finished with reward 109.0. Running Avg 129.8
Episode 153 finished with reward 172.0. Running Avg 129.52
Episode 154 finished with reward 200.0. Running Avg 130.57
Episode 155 finished with reward 200.0. Running Avg 131.38
Episode 156 finished with reward 168.0. Running Avg 131.37
Episode 157 finished with reward 153.0. Running Avg 131.82
Episode 158 finished with reward 200.0. Running Avg 133.09
Episode 159 finished with reward 200.0. Running Avg 133.09
Episode 160 finished with reward 200.0. Running Avg 133.09
Episode 161 finished with reward 200.0. Running Avg 133.09
Episode 162 finished with reward 200.0. Running Avg 133.0

Episode 297 finished with reward 185.0. Running Avg 194.17
Episode 298 finished with reward 200.0. Running Avg 194.72
Episode 299 finished with reward 200.0. Running Avg 194.72
Episode 300 finished with reward 200.0. Running Avg 194.72
Episode 301 finished with reward 200.0. Running Avg 194.95
Episode 302 finished with reward 176.0. Running Avg 194.94
Episode 303 finished with reward 200.0. Running Avg 195.47
CPU times: user 2.33 s, sys: 324 ms, total: 2.65 s
Wall time: 2.42 s


## Final Thoughts

You may be surpised at how quickly this learns, considering it starts with random weights and doesn't get any positive reward until it 'stumbles upon' a victory.

However keep in mind that just because it isn't winning doesn't mean it isn't learning. With every failure it's learning as well. RL can be summed up as follows:

*Try something. If it works, do more of it in the future (update weights to make that outcome more likely). If it doesn't work, do less of it in the future (update weights to make that outcome less likely).*

### Future Work

- Make tf.keras version
- Manual backprop is fine for a single layer network, but if you want to experiment with more complex policy networks auto-backprop is almost a neccesity.
- Use model.predict, engineer labels, then use model.fit (predicting twice so may slow down a bit)